In [8]:
import pandas as pd
from pprint import pprint
from functools import reduce
import re
import os


WEEK_DAYS = ['MON', 'TUE', 'WED', 'THU', 'FRI']
TIME_LIST=[8,9,10,11,12,13,14,15,16,17,18,19]

FINANCE_MEMBERS = ['18XXX9999', '17XXX9999']

INPUT_FILE = os.path.join("Input","Manager2.csv")


In [9]:

free_stu = {}
free_stu2 = {}
free_stu3 = {}
dict_Reg_Name={}

#############    Update dict_Reg_Name

with open(INPUT_FILE, 'r') as tt:
    for line in tt:
        if line.startswith("\"2018/12") or line.startswith("\"2019"):
            pass
        else:
            continue
        #print(line)
        words = line.split(',')
        if words[1][1:-1] not in dict_Reg_Name.keys():
            dict_Reg_Name[words[1][1:-1]]= words[2][1:-1]
print(dict_Reg_Name)
print(len(dict_Reg_Name))




{'99XXX9999': 'Name1', '17XXX9999': 'Name2', '18XXX9999': 'Name3'}
3


In [10]:
####### preprocess slot_csv files
def replace_to_tab(file1, file2):
        c=""
        f = open(file1,'r')
        for lines in f:
                c+=lines.replace(',','\t')
        f.close()

        f2 = open(file2,'w')
        f2.write(c)
        f2.close()

IN_FILE1 = os.path.join("Processed","demo.csv")
OUT_FILE1 = os.path.join("Processed","demox.csv")
IN_FILE2 = os.path.join("Processed","demo2.csv")
OUT_FILE2 = os.path.join("Processed","demox2.csv")

replace_to_tab(IN_FILE1,OUT_FILE1)      #theory_slots
replace_to_tab(IN_FILE2,OUT_FILE2)    #lab_slots


##### Read files
slot = pd.read_csv(OUT_FILE1, sep="\t",index_col = False)
df_theory = pd.DataFrame(slot)
slotx2 = pd.read_csv(OUT_FILE2, sep="\t",index_col = False)
df_lab = pd.DataFrame(slotx2)

   


In [11]:


dict_dayTime_Freereg = {}
dict_dayReg_freeTime = {}
dict_dayName_freeTime = {}

# Iterate through each day
for day in range(0,5):
        dict_freeTime_reg1 = {}         #theory
        dict_freeTime_reg2 = {}         #lab
        dict_reg_freeTime = {}          #theory + lab
        for i in TIME_LIST:
                free_stu[i]= []
                free_stu2[i]= []        
                free_stu3[i]= []


        ############-------------- THEORY FREE SLOT

        df1= df_theory.loc[df_theory['Day'] == day]
        l=list(df1)
        #print(l)
        for i in range(2,len(l)):
                reg= l[i]
                df2 = df1[['Time',reg]]
                #print(df2)
                listtime = df2.loc[df2[reg].astype(object)=='0']['Time'].tolist()
                for j in listtime:
                        free_stu[j].append(dict_Reg_Name[reg])
                dict_freeTime_reg1[reg] = listtime

        #print(dict_freeTime_reg1)


        #############-------------  LAB FREE SLOT

        df1= df_lab.loc[df_lab['Day'] == day]
        l=list(df1)
        for i in range(2,len(l)):
                reg= l[i]
                #print(reg + " : " + dict_Reg_Name[reg])
                df2 = df1[['Time',reg]]
                #print(df2)
                listtime = df2.loc[df2[reg].astype(object)=='0']['Time'].tolist()
                for j in listtime:
                        free_stu2[j].append(dict_Reg_Name[reg])
                        
                dict_freeTime_reg2[reg] = listtime

        #print(dict_freeTime_reg2)

        ################-------------- LAB + THEORY FREE people at specific time
        for j in free_stu:
                for i in free_stu[j]:
                        if (i in free_stu2[j]):
                                free_stu3[j].append(i.title())
        print()
        print()
        print("##################################################################################")
        print(WEEK_DAYS[day])
        print("##################################################################################")
        print()
        pprint(free_stu3)   
        dict_dayTime_Freereg[WEEK_DAYS[day]] = free_stu3.copy()
        #print(dict_dayTime_Freereg)
         
         
        ###############----------   LAB + THEORY FREE slot of each person
        for regno in dict_freeTime_reg1:
                for i in dict_freeTime_reg1[regno]:
                        if(i in dict_freeTime_reg2[regno]):
                                if regno not in dict_reg_freeTime:
                                        dict_reg_freeTime[regno] = []

                                dict_reg_freeTime[regno].append(str(i))

        # Name : Free slots
        print()
        print(WEEK_DAYS[day])      
        print()
        
        dict_reg_freeTime2 = {}
        for k in dict_reg_freeTime:
                print(dict_Reg_Name[k] + " : ",end="")
                print(dict_reg_freeTime[k])
                dict_reg_freeTime2[dict_Reg_Name[k]] = dict_reg_freeTime[k]
                
        dict_dayName_freeTime[WEEK_DAYS[day]] = dict_reg_freeTime2.copy()
        dict_dayReg_freeTime[WEEK_DAYS[day]] = dict_reg_freeTime.copy()





##################################################################################
MON
##################################################################################

{8: ['Name1', 'Name3'],
 9: [],
 10: ['Name1', 'Name3'],
 11: ['Name1', 'Name3', 'Name2'],
 12: ['Name1', 'Name3'],
 13: ['Name1', 'Name3', 'Name2'],
 14: ['Name2'],
 15: ['Name2'],
 16: ['Name2'],
 17: [],
 18: ['Name2'],
 19: ['Name2']}

MON

Name1 : ['8', '10', '11', '12', '13']
Name2 : ['11', '13', '14', '15', '16', '18', '19']
Name3 : ['8', '10', '11', '12', '13']


##################################################################################
TUE
##################################################################################

{8: ['Name2'],
 9: [],
 10: ['Name1', 'Name3'],
 11: ['Name1', 'Name3'],
 12: ['Name1', 'Name3', 'Name2'],
 13: ['Name1', 'Name3', 'Name2'],
 14: [],
 15: [],
 16: ['Name2'],
 17: ['Name2'],
 18: ['Name2'],
 19: ['Name1', 'Name3', 'Name2']}

TUE

Name1 : ['10', '11', '12', '13', '1

In [12]:
#WEEK_DAYS : TIME : FREE_PERSON 

pprint(dict_dayTime_Freereg)

#Write to file
from pprint import pformat
OUTPUT_FILE = os.path.join("Output","out_dtp.txt")
with open(OUTPUT_FILE, "w") as fout:
    fout.write(pformat(dict_dayTime_Freereg))

{'FRI': {8: [],
         9: [],
         10: ['Name1', 'Name3'],
         11: ['Name2'],
         12: ['Name1', 'Name3'],
         13: ['Name1', 'Name3', 'Name2'],
         14: ['Name2'],
         15: ['Name2'],
         16: [],
         17: [],
         18: ['Name1', 'Name3', 'Name2'],
         19: ['Name1', 'Name3', 'Name2']},
 'MON': {8: ['Name1', 'Name3'],
         9: [],
         10: ['Name1', 'Name3'],
         11: ['Name1', 'Name3', 'Name2'],
         12: ['Name1', 'Name3'],
         13: ['Name1', 'Name3', 'Name2'],
         14: ['Name2'],
         15: ['Name2'],
         16: ['Name2'],
         17: [],
         18: ['Name2'],
         19: ['Name2']},
 'THU': {8: [],
         9: ['Name2'],
         10: ['Name1', 'Name3'],
         11: ['Name1', 'Name3'],
         12: ['Name1', 'Name3'],
         13: ['Name1', 'Name3', 'Name2'],
         14: ['Name2'],
         15: [],
         16: [],
         17: ['Name2'],
         18: ['Name1', 'Name3', 'Name2'],
         19: ['Name1', 'Name3

In [13]:
#WEEK_DAYS : PERSON : FREE_TIME

pprint(dict_dayName_freeTime)

#Write to file
from pprint import pformat
OUTPUT_FILE = os.path.join("Output","out_dpt.txt")
with open(OUTPUT_FILE, "w") as fout:
    fout.write(pformat(dict_dayReg_freeTime))

{'FRI': {'Name1': ['10', '12', '13', '18', '19'],
         'Name2': ['11', '13', '14', '15', '18', '19'],
         'Name3': ['10', '12', '13', '18', '19']},
 'MON': {'Name1': ['8', '10', '11', '12', '13'],
         'Name2': ['11', '13', '14', '15', '16', '18', '19'],
         'Name3': ['8', '10', '11', '12', '13']},
 'THU': {'Name1': ['10', '11', '12', '13', '18', '19'],
         'Name2': ['9', '13', '14', '17', '18', '19'],
         'Name3': ['10', '11', '12', '13', '18', '19']},
 'TUE': {'Name1': ['10', '11', '12', '13', '19'],
         'Name2': ['8', '12', '13', '16', '17', '18', '19'],
         'Name3': ['10', '11', '12', '13', '19']},
 'WED': {'Name1': ['8', '9', '11', '12', '13', '18', '19'],
         'Name2': ['11', '13', '14', '15', '19'],
         'Name3': ['8', '9', '11', '12', '13', '18', '19']}}


In [14]:
# UTILITIES

###############------------ Find common free slot of students:

def common_free_timeOf(list_members, given_day = None):
    if given_day is not None: 
        print("Free common time of members on {} : ".format(given_day))
        dict_reg_freeTime = dict_dayReg_freeTime[given_day]
        newDict = {}
        for regno in dict_reg_freeTime:
                if regno in list_members:
                        newDict[regno] = dict_reg_freeTime[regno]
                        
        required_list = sorted(list(reduce(set.intersection, [set(item) for item in list(newDict.values()) ])))
        print(required_list)
        return required_list

    else:
        required_dict = {}
        print("Free common time of members : ")
        for day in dict_dayReg_freeTime:
            print("For " + day, end = " : ")
            dict_reg_freeTime = dict_dayReg_freeTime[day]
            newDict = {}
            for regno in dict_reg_freeTime:
                if regno in list_members:
                    newDict[regno] = dict_reg_freeTime[regno]
            required_list = sorted(list(reduce(set.intersection, [set(item) for item in list(newDict.values()) ])))
            print(required_list)
            required_dict[day] = required_list
        return required_dict

        
###############------------ Find free time of specific student:

def free_timeOf(member, given_day = None):
    if given_day is not None: 
        print("Free time of {} on {} is : ".format(member, given_day))
        required_list = dict_dayReg_freeTime[given_day][member]
        print(required_list)
        return required_list
    else:
        required_dict = {}
        print("Free time of {} is : ".format(member))
        for day in dict_dayReg_freeTime:
            print("For {}".format(day), end=" : ")
            required_list = dict_dayReg_freeTime[day][member]
            print(required_list)
            required_dict[day] = required_list
        return required_dict
        
###############------------ Find students free at given time:

        
def students_freeAt(given_time, given_day = None):
    #pprint(dict_dayTime_Freereg)
    if given_day is not None:   
        print("Students free at {}:00 on {} are : ".format(given_time, given_day))
        required_list = dict_dayTime_Freereg[given_day][given_time] 
        print(required_list)   
        
    else:
        required_dict = {}
        print("Students free at {} are : ".format(given_time))
        for day in dict_dayTime_Freereg:
            print("For {}".format(day), end=" : ")
            required_list = dict_dayTime_Freereg[day][given_time]
            print(required_list)
            required_dict[day] = required_list
        return required_dict
    
students_freeAt(8)
print()
students_freeAt(8, 'WED')
print()
free_timeOf('18XXX9999')
print()
free_timeOf('18XXX9999', 'WED')
print()
common_free_timeOf(FINANCE_MEMBERS)
print()
common_free_timeOf(FINANCE_MEMBERS,'WED')

Students free at 8 are : 
For THU : []
For MON : ['Name1', 'Name3']
For WED : ['Name1', 'Name3']
For FRI : []
For TUE : ['Name2']

Students free at 8:00 on WED are : 
['Name1', 'Name3']

Free time of 18XXX9999 is : 
For THU : ['10', '11', '12', '13', '18', '19']
For MON : ['8', '10', '11', '12', '13']
For WED : ['8', '9', '11', '12', '13', '18', '19']
For FRI : ['10', '12', '13', '18', '19']
For TUE : ['10', '11', '12', '13', '19']

Free time of 18XXX9999 on WED is : 
['8', '9', '11', '12', '13', '18', '19']

Free common time of members : 
For THU : ['13', '18', '19']
For MON : ['11', '13']
For WED : ['11', '13', '19']
For FRI : ['13', '18', '19']
For TUE : ['12', '13', '19']

Free common time of members on WED : 
['11', '13', '19']


['11', '13', '19']